# imports

In [2]:
import pandas as pd
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt; plt.rcParams['font.family'] = 'Malgun Gothic'
import seaborn as sns; sns.set(font='Malgun Gothic')

from scipy.stats import skew, kurtosis

import warnings; warnings.filterwarnings(action='ignore')
from IPython.core.display import display, HTML
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

In [3]:
from sklearn import datasets

# sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

# preprocessing
from sklearn.impute import SimpleImputer, MissingIndicator, KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.preprocessing import Binarizer
from sklearn.decomposition import PCA

# estimators
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from xgboost import XGBClassifier, XGBRegressor, XGBRFClassifier, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

# neural
import tensorflow as tf; tf.random.set_seed(0)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ensemble
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, VotingClassifier, VotingRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor

# metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error

# curves
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV

# tools
import pycaret as pyc
import optuna

In [4]:
import datetime as dt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams['figure.figsize'] = (8, 4)
mpl.rcParams['axes.grid'] = True

# read

In [5]:
train = pd.read_csv('./input/train.csv')
test  = pd.read_csv('./input/test.csv')
sub   = pd.read_csv('./input/sample_submission.csv')

In [6]:
train.shape, test.shape

((1205, 12), (50, 10))

In [7]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,"모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐어채무침 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,"모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌개 시래기조림 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이 (배추,고추가루:국내산)","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무침 바나나 포기김치 (배추,고추가루:국내산)",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,"모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국 느타리호박볶음 포기김치 (배추,고추가루:국내산)","카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 견과류조림 요구르트 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음 포기김치 (배추,고추가루:국내산)",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국 멸치볶음 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 요구르트 포기김치 (배추,고추가루:국내산)","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포기김치 (배추,고추가루:국내산)",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,"모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방풍나물 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채 청경채무침 요구르트 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:국내산) 감자소세지볶음 콩나물무침 포기김치 (배추,고추가루:국내산)",925.0,330.0


In [8]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자전 양상추샐러드/사과 포기김치,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추샐러드*딸기D,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 양상추샐러드 포기김치,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상추샐러드*오미자D,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양상추샐러드 포기김치,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상추샐러드*파인요거트D,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완자전 양상추샐러드 포기김치,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치 양상추샐러드*망고D,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양상추샐러드 포기김치,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸딩 양상추샐러드*오리엔탈D,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [10]:
train['일자'].min(), train['일자'].max()

('2016-02-01', '2021-01-26')

In [11]:
test['일자'].min(), test['일자'].max()

('2021-01-27', '2021-04-09')

# df concat

In [12]:
df = pd.concat([train, test])
df.shape

(1255, 12)

# casting

In [13]:
df['현본사소속재택근무자수'] = df['현본사소속재택근무자수'].astype('int64')

# derived feat

In [14]:
df['출근자수'] = df['본사정원수'] - df[['본사휴가자수', '본사출장자수', '현본사소속재택근무자수']].sum(axis=1)

In [15]:
df['출근비율'] = df['출근자수']    / df['본사정원수']
df['휴가비율'] = df['본사휴가자수'] / df['본사정원수']
df['출장비율'] = df['본사출장자수'] / df['본사정원수']
df['야근비율'] = df['본사시간외근무명령서승인건수'] / df['본사정원수']
df['재택비율'] = df['현본사소속재택근무자수']      / df['본사정원수']

In [16]:
df['중식특식'] = np.where(df['중식메뉴'].str.contains('특식'), 1, 0)

In [17]:
df['자기계발'] = np.where((df['석식메뉴'].str.len() <= 20) & (df['출근자수'] > 0), 1, 0)

In [18]:
drop_cols = ['요일', '출근자수', '조식메뉴', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', ]
df.drop(drop_cols, axis=1, inplace=True)

# datetime

In [19]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib import parse

In [20]:
df['일자'] = pd.to_datetime(df['일자'])

In [21]:
# 데이터의 날짜 범위
start_d  = pd.to_datetime(df['일자'].min())
end_d    = pd.to_datetime(df['일자'].max())
dates    = pd.date_range(start_d, end_d, freq='D')
dates_df = pd.DataFrame({'날짜': dates})
dates_df.head()

,날짜
0,2016-02-01
1,2016-02-02
2,2016-02-03
3,2016-02-04
4,2016-02-05


In [22]:
# 공휴일 api
# https://www.data.go.kr/iim/api/selectAPIAcountView.do
# http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear=2019&solMonth=03&ServiceKey=서비스키

def getHoliDeInfo(start_year=2016, end_year=2021):
    
    """공공데이터포털에서 공휴일 OpenAPI 불러오기"""
    
    key = parse.unquote('uERvRwcxuakpKoIT7D%2FVwKIosqibQV%2BLsNcj6uKMdKBPiiy9tpNKFKg8P%2Bs0DqEuiz%2BrFgivpo4icWJzh27h5A%3D%3D')
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo'
    
    list = []
    for year in range(start_year, end_year + 1):
        
        params   = {"ServiceKey": key, "solYear": year, "numOfRows": 100}
        response = requests.get(url, params=params)
        xml      = BeautifulSoup(response.text, 'lxml')
        
        dict = {}
        for i in xml.find('items'):
            dict = {
                '날짜': datetime.strptime(i.find('locdate').text.strip(), '%Y%m%d'), 
                '휴일': i.find('isholiday').text.strip(), 
                '종류': i.find('datekind').text.strip(), 
                '이름': i.find('datename').text.strip(), 
            }
            list.append(dict)
        
        df = pd.DataFrame(list)
        df = df.fillna('')
        
        df['휴일'].replace('Y', 1, inplace=True)
        df['휴일'].replace('N', 0, inplace=True)
        df['휴일'].fillna(0)
        df['휴일'] = df['휴일'].astype('int64')
    
    return df

In [22]:
start_y = pd.to_datetime(start_d).year
end_y   = pd.to_datetime(end_d).year

holiday_df = getHoliDeInfo(start_y, end_y)

In [41]:
holiday_df

,날짜,휴일,종류,이름
0,2016-01-01,1,01,신정
1,2016-02-07,1,01,설날
2,2016-02-08,1,01,설날
3,2016-02-09,1,01,설날
4,2016-02-10,1,01,대체공휴일
...,...,...,...,...
107,2021-10-03,1,01,개천절
108,2021-10-04,1,01,대체공휴일
109,2021-10-09,1,01,한글날
110,2021-10-11,1,01,대체공휴일


In [54]:
calendar_df = pd.merge(dates_df, holiday_df[holiday_df['종류'] == '01'], how='left', on='날짜')[['날짜', '휴일']]
calendar_df.fillna(0, inplace=True)
calendar_df['휴일'] = calendar_df['휴일'].astype('int64')
calendar_df.head()

,날짜,휴일
0,2016-02-01,0
1,2016-02-02,0
2,2016-02-03,0
3,2016-02-04,0
4,2016-02-05,0


In [55]:
# 날짜 파생변수
calendar_df['년']   = calendar_df['날짜'].dt.year
calendar_df['월']   = calendar_df['날짜'].dt.month
calendar_df['일']   = calendar_df['날짜'].dt.day
calendar_df['주']   = calendar_df['날짜'].dt.week
calendar_df['요일'] = calendar_df['날짜'].dt.weekday

In [56]:
# 주말 == 휴일
calendar_df.loc[(calendar_df['요일'] == 5) | (calendar_df['요일'] == 6), '휴일'] = 1

In [57]:
# 코로나 발생
calendar_df['코로나'] = calendar_df['년'].apply(lambda x: 1 if x >= 2020 else 0)

In [58]:
calendar_df.head()

,날짜,휴일,년,월,일,주,요일,코로나
0,2016-02-01,0,2016,2,1,5,0,0
1,2016-02-02,0,2016,2,2,5,1,0
2,2016-02-03,0,2016,2,3,5,2,0
3,2016-02-04,0,2016,2,4,5,3,0
4,2016-02-05,0,2016,2,5,5,4,0
